In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import tensorflow as tf
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import cohen_kappa_score, log_loss
from sklearn.metrics import classification_report
import pandas as pd



# Veriyi yükleme
data = pd.read_excel('veriseti_tez.xlsx')

# Sütunları seçme
columns_to_process = ["drugname", "indi_pt", "prod_ai", "pt"]

# Sütunlardaki veriyi küçük harfe dönüştürme ve boşlukları temizleme
for col in columns_to_process:
    data[col] = data[col].str.lower().str.strip().str.replace(r'\s+', ' ', regex=True)

# Benzersiz değerlerin ve bunların kaç defa geçtiğinin hesaplanması
for col in columns_to_process:
    print(f"\n{col} sütunundaki benzersiz değerler ve frekansları:")
    print(data[col].value_counts())



C:\Users\HP\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:241: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")



drugname sütunundaki benzersiz değerler ve frekansları:
drugname
folic acid                                       1339
furosemide                                       1218
acetaminophen                                    1187
sodium chloride                                  1161
aspirin 81 mg chewable tablet                    1152
                                                 ... 
pantoprazole 40 mg tablet,delayed release           1
acetaminophen 325 mg capsule                        1
donepezil 5 mg tablet                               1
dulcolax (bisacodyl) 10 mg rectal suppository       1
zolpidem tartrate                                   1
Name: count, Length: 1182, dtype: int64

indi_pt sütunundaki benzersiz değerler ve frekansları:
indi_pt
asthma                                   6480
chronic obstructive pulmonary disease    6144
pneumonia                                6144
double hit lymphoma                      2304
diffuse large b-cell lymphoma            1408
      

In [3]:
# Drugname'e göre gruplama ve sınıfları birleştirme
drugname_to_pt = data.groupby('drugname')['pt'].apply(lambda x: '|'.join(set(x))).reset_index()

# Yeni sınıf bilgilerini orijinal veri setine ekleme
data = data.merge(drugname_to_pt, on='drugname', suffixes=('', '_merged'))

# Orijinal pt sütununu yeni pt ile değiştirme
data['pt'] = data['pt_merged']
data.drop(columns=['pt_merged'], inplace=True)

# Güncellenmiş sınıf sayısı
updated_class_count = data['pt'].nunique()
print(f"Güncellenmiş sınıf sayısı: {updated_class_count}")



Güncellenmiş sınıf sayısı: 635


In [6]:
# Herhangi bir sütunun eşleşmesine göre grup anahtarı oluşturma
# Veriyi yükleme
data = pd.read_excel('veriseti_tez.xlsx')

# Sütunları seçme
columns_to_process = ["drugname", "indi_pt", "prod_ai", "pt"]

# Sütunlardaki veriyi küçük harfe dönüştürme ve boşlukları temizleme
for col in columns_to_process:
    data[col] = data[col].str.lower().str.strip().str.replace(r'\s+', ' ', regex=True)
    
# Herhangi bir sütunun eşleşmesine göre grup anahtarı oluşturma
def create_group_key(row, data, columns):
    for col in columns:
        # Aynı sütundaki diğer değerlerle karşılaştırma
        if row[col] in data[col].values:
            return f"{col}_{row[col]}"
    return None

# Grup anahtarlarını oluştur
group_columns = ['prod_ai', 'drugname', 'indi_pt']
data['group_key'] = data.apply(lambda row: create_group_key(row, data, group_columns), axis=1)

# Grup anahtarına göre sınıfları birleştirme
grouped = data.groupby('group_key')['pt'].apply(lambda x: '|'.join(set(x))).reset_index()

# Birleştirilen sınıfları orijinal veri setine ekleme
data = data.merge(grouped, on='group_key', suffixes=('', '_merged'))

# Orijinal pt sütununu güncelle
data['pt'] = data['pt_merged']
data.drop(columns=['pt_merged', 'group_key'], inplace=True)

# Güncellenmiş sınıf sayısı
updated_class_count = data['pt'].nunique()
print(f"Güncellenmiş sınıf sayısı: {updated_class_count}")

# Yeni target sınıflarını liste olarak görüntüleme
new_classes = data['pt'].unique()
print(f"Yeni target sınıflar:\n{new_classes}")

# Sınıfların frekanslarını görüntüleme
class_frequencies = data['pt'].value_counts()
print(f"Sınıf frekansları:\n{class_frequencies}")



C:\Users\HP\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:241: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Güncellenmiş sınıf sayısı: 599
Yeni target sınıflar:
['diarrhoea|therapy interrupted|illness|sickle cell anaemia with crisis|hospitalisation|product use issue|pain|pneumonia'
 'drug level increased|therapy interrupted|death|small intestinal obstruction|loss of consciousness|infection|pneumatosis intestinalis|cardiac failure|urinary tract infection|therapy cessation|pyrexia|coronavirus infection|acute kidney injury|covid-19|nausea|platelet count decreased|asthma|influenza|complications of transplant surgery|hypervolaemia|pneumonia|chills|diarrhoea|kidney infection|intestinal ischaemia|intensive care|neuropathy peripheral|respiratory distress|cystitis|vomiting|ulcer|hypotension|infection in an immunocompromised host|septic shock'
 'therapy interrupted|death|infection|urinary tract infection|blood pressure abnormal|pyrexia|large intestine polyp|nausea|abdominal pain upper|asthma|influenza|colitis|haemorrhoids|pneumonia|chills|diarrhoea|kidney infection|respiratory distress|cystitis|rectal

In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import tensorflow as tf
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import cohen_kappa_score, log_loss
from sklearn.metrics import classification_report

# Veri yükleme ve hazırlama
#data = pd.read_excel('veriseti_tez.xlsx')

X = data[["prod_ai", "drugname", "indi_pt", "primaryid", "caseid", "age", 
          "event_dt", "init_fda_dt", "fda_dt", "mfr_dt", "rept_dt", "wt", 
          "start_dt", "outc_cod", "occp_cod", "drug_seq"]].copy()
y = data['pt'].copy()

# Veri ön işleme
numeric_cols = X.select_dtypes(include=[np.number]).columns
categorical_cols = X.select_dtypes(exclude=[np.number]).columns

X.loc[:, numeric_cols] = X[numeric_cols].fillna(X[numeric_cols].mean())
for col in categorical_cols:
    X.loc[:, col] = X[col].fillna(X[col].mode()[0])
    X.loc[:, col] = LabelEncoder().fit_transform(X[col])

# Target encoding ve veri türü dönüşümü
le = LabelEncoder()
y_encoded = le.fit_transform(y).astype('float32')
num_classes = len(np.unique(y_encoded))

# Standardizasyon
scaler = StandardScaler()
X.loc[:, numeric_cols] = scaler.fit_transform(X[numeric_cols])
X = X.astype('float32')

# Veri bölme
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Tensor dönüşümü
X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)

# Model
model = Sequential([
    Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.4),
    Dense(128, activation='relu'),
    Dropout(0.4),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
])

# Derleme ve eğitim
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

history = model.fit(X_train, 
                   y_train,
                   epochs=50,
                   batch_size=32,
                   validation_split=0.2,
                   verbose=1)
# Model değerlendirme
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)



# Metrikler
print(f"Accuracy: {accuracy_score(y_test, y_pred_classes):.4f}")
print(f"Macro F1: {f1_score(y_test, y_pred_classes, average='macro'):.4f}")
print(f"Weighted F1: {f1_score(y_test, y_pred_classes, average='weighted'):.4f}")
print(f"Cohen's Kappa: {cohen_kappa_score(y_test, y_pred_classes):.4f}")
#print(f"Log Loss: {log_loss(y_test_cat, y_pred):.4f}")
print("\nDetaylı Sınıflandırma Raporu:")
print(classification_report(y_test, y_pred_classes))

C:\Users\HP\AppData\Local\Temp\ipykernel_11404\3910121473.py:36: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[-2.21662163 -2.21333397 -2.21333397 ...  1.23352587  1.23352587
  1.23352587]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  X.loc[:, numeric_cols] = scaler.fit_transform(X[numeric_cols])
C:\Users\HP\AppData\Local\Temp\ipykernel_11404\3910121473.py:36: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[-2.21662163 -2.21333397 -2.21333397 ...  1.23352587  1.23352587
  1.23352587]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  X.loc[:, numeric_cols] = scaler.fit_transform(X[numeric_cols])
C:\Users\HP\AppData\Local\Temp\ipykernel_11404\3910121473.py:36: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pa

Epoch 1/50
882/882 [==============================] - 6s 6ms/step - loss: 5.6465 - accuracy: 0.2916 - val_loss: 4.4195 - val_accuracy: 0.3430
Epoch 2/50
882/882 [==============================] - 5s 6ms/step - loss: 4.1047 - accuracy: 0.3408 - val_loss: 3.5972 - val_accuracy: 0.3546
Epoch 3/50
882/882 [==============================] - 5s 6ms/step - loss: 3.6438 - accuracy: 0.3527 - val_loss: 3.2296 - val_accuracy: 0.3822
Epoch 4/50
882/882 [==============================] - 5s 6ms/step - loss: 3.3062 - accuracy: 0.3781 - val_loss: 2.9765 - val_accuracy: 0.4090
Epoch 5/50
882/882 [==============================] - 5s 6ms/step - loss: 3.0919 - accuracy: 0.3948 - val_loss: 2.7423 - val_accuracy: 0.4215
Epoch 6/50
882/882 [==============================] - 5s 6ms/step - loss: 2.9279 - accuracy: 0.4100 - val_loss: 2.5819 - val_accuracy: 0.4478
Epoch 7/50
882/882 [==============================] - 5s 6ms/step - loss: 2.8143 - accuracy: 0.4210 - val_loss: 2.5756 - val_accuracy: 0.4413
Epoch 

C:\Users\HP\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\HP\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\HP\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\HP\anaconda3\Lib\site-packages\sklearn\metrics\_

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from sklearn.metrics import accuracy_score, f1_score, cohen_kappa_score, classification_report
from sklearn.metrics import accuracy_score, f1_score, cohen_kappa_score, matthews_corrcoef
from tensorflow.keras.layers import BatchNormalization


#Normal veriseti (Yan etki sütununa hiç dokunmadığım ) 
# Veri yükleme ve hazırlama
data = pd.read_excel('veriseti_tez.xlsx')


X = data[["prod_ai", "drugname", "indi_pt", "primaryid", "caseid", "age", 
          "event_dt", "init_fda_dt", "fda_dt", "mfr_dt", "rept_dt", "wt", 
          "start_dt", "outc_cod", "occp_cod", "drug_seq"]].copy()



y = data['pt'].copy()

# Veri ön işleme
numeric_cols = X.select_dtypes(include=[np.number]).columns
categorical_cols = X.select_dtypes(exclude=[np.number]).columns

X.loc[:, numeric_cols] = X[numeric_cols].fillna(X[numeric_cols].mean())
for col in categorical_cols:
    X.loc[:, col] = X[col].fillna(X[col].mode()[0])
    X.loc[:, col] = LabelEncoder().fit_transform(X[col])

#encoding ve veri türü dönüşümü
le = LabelEncoder()
y_encoded = le.fit_transform(y).astype('float32')
num_classes = len(np.unique(y_encoded))

# Standardizasyon
scaler = StandardScaler()
X.loc[:, numeric_cols] = scaler.fit_transform(X[numeric_cols])
X = X.astype('float32')

# Veri bölme
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Tensor dönüşümü
X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)

# Model Yapısını Geliştirme
model = Sequential([
    Dense(512, activation='relu', input_shape=(X_train.shape[1],)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.4),
    Dense(num_classes, activation='softmax')
])

# Modeli derleme
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
# Erken Durdurma
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

# Öğrenme Oranı İyileştirme
from tensorflow.keras.callbacks import LearningRateScheduler

def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

lr_scheduler = LearningRateScheduler(scheduler)

# Model Eğitimi
history = model.fit(X_train, 
                    y_train,
                    epochs=100,
                    batch_size=64,
                    validation_split=0.2,
                    callbacks=[early_stopping, lr_scheduler],
                    verbose=1)

# Model değerlendirme
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Metrikler
print(f"Accuracy: {accuracy_score(y_test, y_pred_classes):.4f}")
print(f"Macro F1: {f1_score(y_test, y_pred_classes, average='macro'):.4f}")
print(f"Weighted F1: {f1_score(y_test, y_pred_classes, average='weighted'):.4f}")
print(f"Cohen's Kappa: {cohen_kappa_score(y_test, y_pred_classes):.4f}")
print(f"Matthews Correlation Coefficient (MCC): {matthews_corrcoef(y_test, y_pred_classes):.4f}")
print("\nDetaylı Sınıflandırma Raporu:")
print(classification_report(y_test, y_pred_classes))


C:\Users\HP\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:241: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\HP\AppData\Local\Temp\ipykernel_12928\4246677128.py:48: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[-2.21662163 -2.21333397 -2.21333397 ...  1.23352587  1.23352587
  1.23352587]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  X.loc[:, numeric_cols] = scaler.fit_transform(X[numeric_cols])
C:\Users\HP\AppData\Local\Temp\ipykernel_12928\4246677128.py:48: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[-2.21662163 -2.21333397 -2.21333397 ...  1.23352587  1.23352587
  1.23352587]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  X.loc[:, nume

Epoch 1/100
441/441 [==============================] - 6s 10ms/step - loss: 5.0537 - accuracy: 0.0717 - val_loss: 4.0589 - val_accuracy: 0.0944 - lr: 0.0010
Epoch 2/100
441/441 [==============================] - 4s 10ms/step - loss: 3.9900 - accuracy: 0.0971 - val_loss: 3.7639 - val_accuracy: 0.1046 - lr: 0.0010
Epoch 3/100
441/441 [==============================] - 4s 10ms/step - loss: 3.7779 - accuracy: 0.1057 - val_loss: 3.5600 - val_accuracy: 0.1093 - lr: 0.0010
Epoch 4/100
441/441 [==============================] - 4s 9ms/step - loss: 3.6545 - accuracy: 0.1075 - val_loss: 3.5004 - val_accuracy: 0.1115 - lr: 0.0010
Epoch 5/100
441/441 [==============================] - 4s 10ms/step - loss: 3.5421 - accuracy: 0.1095 - val_loss: 3.3145 - val_accuracy: 0.1224 - lr: 0.0010
Epoch 6/100
441/441 [==============================] - 4s 10ms/step - loss: 3.4074 - accuracy: 0.1182 - val_loss: 3.2397 - val_accuracy: 0.1315 - lr: 0.0010
Epoch 7/100
441/441 [==============================] - 4s 1

Epoch 52/100
441/441 [==============================] - 4s 10ms/step - loss: 2.5262 - accuracy: 0.1739 - val_loss: 2.4512 - val_accuracy: 0.1740 - lr: 1.4996e-05
Epoch 53/100
441/441 [==============================] - 4s 10ms/step - loss: 2.5379 - accuracy: 0.1721 - val_loss: 2.4512 - val_accuracy: 0.1733 - lr: 1.3569e-05
Epoch 54/100
441/441 [==============================] - 4s 10ms/step - loss: 2.5308 - accuracy: 0.1739 - val_loss: 2.4501 - val_accuracy: 0.1719 - lr: 1.2277e-05
Epoch 55/100
441/441 [==============================] - 4s 10ms/step - loss: 2.5360 - accuracy: 0.1702 - val_loss: 2.4525 - val_accuracy: 0.1733 - lr: 1.1109e-05
Epoch 56/100
441/441 [==============================] - 4s 10ms/step - loss: 2.5344 - accuracy: 0.1733 - val_loss: 2.4497 - val_accuracy: 0.1733 - lr: 1.0052e-05
Epoch 57/100
441/441 [==============================] - 4s 9ms/step - loss: 2.5361 - accuracy: 0.1732 - val_loss: 2.4501 - val_accuracy: 0.1711 - lr: 9.0953e-06
Epoch 58/100
441/441 [=======

C:\Users\HP\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\HP\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\HP\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\HP\anaconda3\Lib\site-packages\sklearn\metrics\_